# FLOPs

In [62]:
import paddle
# https://github.com/Lyken17/pytorch-OpCounter/tree/master/thop
# https://arxiv.org/pdf/1611.06440.pdf

In [63]:
model = paddle.vision.models.resnet34()

In [64]:
paddle.summary(model, (1, 3, 224, 224))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
    Conv2D-253       [[1, 3, 224, 224]]   [1, 64, 112, 112]        9,408     
  BatchNorm2D-253   [[1, 64, 112, 112]]   [1, 64, 112, 112]         256      
     ReLU-120       [[1, 64, 112, 112]]   [1, 64, 112, 112]          0       
    MaxPool2D-8     [[1, 64, 112, 112]]    [1, 64, 56, 56]           0       
    Conv2D-254       [[1, 64, 56, 56]]     [1, 64, 56, 56]        36,864     
  BatchNorm2D-254    [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
     ReLU-121        [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
    Conv2D-255       [[1, 64, 56, 56]]     [1, 64, 56, 56]        36,864     
  BatchNorm2D-255    [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
  BasicBlock-113     [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
    Conv2D-256       [[1, 64, 56, 56]]     [1, 64, 56, 56]    

{'total_params': 21814696, 'trainable_params': 21780648}

In [91]:
totle_num = 0

def _conv2d_hook(layer, inputs, output):
    '''
    '''
    o = output.shape[0] * output.shape[2] * output.shape[2]   
    k = layer.weight.numel() 
    b = 1 if layer.bias is not None else 0
    
    layer.op_num += o * (k + b)


def _bn2d_hook(layer, inputs, output):
    pass


def _maxpool2d_hook(layer, inputs, output):
    pass


def _avgpool2d_hook(layer, inputs, output):
    pass


def _relu_hook(layer, inputs, output):
    '''
    '''
    layer.op_num += inputs[0].numel()


def _linear_hook(layer, inputs, output):
    '''
    '''
    in_channels = layer.weight.shape[0]    
    layer.op_num += in_channels * output.numel()


hooks = {
    paddle.nn.Conv2D : _conv2d_hook,
    paddle.nn.ReLU : _relu_hook,
    paddle.nn.Linear : _linear_hook,
    paddle.nn.BatchNorm2D : _bn2d_hook,
    
}



def register_hook(model):
    '''
    '''
    for m in model.sublayers():
        if type(m) in hooks:
            m.register_forward_post_hook( hooks[type(m)] )
            m.register_buffer('op_num', paddle.zeros(shape=(1, ), dtype='float32'))
        elif type(m) in (paddle.nn.Sequential, ):
            pass
        else:
            print(f'do not support {type(m)}')
            
    print('register done...')

In [92]:
register_hook(model)

do not support <class 'paddle.nn.layer.pooling.MaxPool2D'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.resnet.BasicBlock'>
do not support <class 'paddle.vision.models.r

In [93]:
model.eval()
model(paddle.rand((1, 3, 224, 224))).shape

[1, 1000]

In [95]:
num_total = 0
for i, m in enumerate(model.sublayers()):
    if hasattr(m, 'op_num'):
        num_total += m.op_num.numpy()[0]

print(num_total)

33005915136.0
